## Load and prepare data

In [122]:
signalPattern,signalOutput = open("signalLog.txt") do file
#signalPattern,signalOutput = open("exampleSignalLog.txt") do file
    pattern = String[]
    output = String[]
    nlines=0
    for line in eachline(file)
        nlines += 1
        splitString = split(line,'|')
        append!(pattern,split(splitString[1]))
        append!(output, split(splitString[2]))
    end
    (reshape(pattern,10,nlines),reshape(output,4,nlines))
end

(["cbdfag" "defgac" … "defcg" "fcde"; "bf" "bfcedag" … "fbeag" "ebafcgd"; … ; "bgdaef" "acbedg" … "agdfbe" "gacefd"; "fgeb" "cdebfa" … "agcedb" "fgceab"], ["bf" "abcfed" … "fgeda" "gcaed"; "defbag" "adfb" … "da" "gec"; "efadgbc" "fdba" … "eafbgd" "cfegba"; "bfgeda" "ab" … "abfd" "fdaegc"])

In [69]:
signalPattern

10×10 Array{String,2}:
 "be"       "edbfga"   "fgaebd"   …  "bdfegc"   "egadfb"   "gcafb"
 "cfbegad"  "begcd"    "cg"          "cbegaf"   "cdbfeg"   "gcf"
 "cbdgef"   "cbg"      "bdaec"       "gecbf"    "cegd"     "dcaebfg"
 "fgaecd"   "gc"       "gdafb"       "dfcage"   "fecab"    "ecagb"
 "cgeb"     "gcadebf"  "agbcfd"      "bdacg"    "cgb"      "gf"
 "fdcge"    "fbgde"    "gdcbef"   …  "ed"       "gbdefca"  "abcdeg"
 "agebfd"   "acbgfd"   "bgcad"       "bedf"     "cg"       "gaef"
 "fecdb"    "abcde"    "gfac"        "ced"      "fgcdab"   "cafbge"
 "fabcd"    "gfcbed"   "gcb"         "adcbefg"  "egfdb"    "fdbac"
 "edb"      "gfec"     "cdgabef"     "gebcd"    "bfceg"    "fegbdc"

In [70]:
signalOutput

4×10 Array{String,2}:
 "fdgacbe"  "fcgedb"   "cg"      "efabcd"  …  "ed"      "gbdfcae"  "fgae"
 "cefdb"    "cgb"      "cg"      "cedba"      "bcgafe"  "bgc"      "cfgab"
 "cefbgd"   "dgebacf"  "fdcagb"  "gadfec"     "cdgba"   "cg"       "fg"
 "gcbe"     "gc"       "cbg"     "cb"         "cbgef"   "cgb"      "bagce"

## Part 1

In [23]:
count(x->(length(x)==2||length(x)==3||length(x)==4||length(x)==7),signalOutput)

532

## Part 2

In [118]:
function buildDecoder(input)
    
    joinedInput = join(input)
    
    #count all occurences of letters a through g
    letters = ['a','b','c','d','e','f','g']
    numLetters = Array{Int}(undef,7)
    for (i,c) in enumerate(letters)
        numLetters[i]=count(x->x==c,joinedInput)
    end
    
    
    decoder = Dict{Char,Char}() #dictionary to be filled with old - new mapping
    #dictionary containing occurence rate as key and letters as values excluding tied values 7 and 8
    occurenceReference = Dict(4=>'e', 6=>'b',9=>'f') 
    
    mapFour = filter(x->length(x)==4,input)[1] #find the representation of 4 (has length 4)
    
    for i in 1:7  
        #tiebreak 7 = dg as g is not part of 4
        if (numLetters[i] == 7)
            if isnothing(findfirst(letters[i],mapFour)) 
                decoder[letters[i]] = 'g'
            else
                decoder[letters[i]] = 'd'
            end
        elseif(numLetters[i] == 8)
        #tiebreak 8 = ac as a is not part of 4
            if isnothing(findfirst(letters[i],mapFour))
                decoder[letters[i]] = 'a'
            else
                decoder[letters[i]] = 'c'
            end
        else
            decoder[letters[i]] = occurenceReference[numLetters[i]]
        end
    end
    return decoder
end
   
function decodeOutput(output,decoder)
    numbers = Array{Int}(undef,4)
    reference = Dict("abcefg"=>0 ,"cf"=>1, "acdeg"=>2, "acdfg"=>3, 
    "bcdf"=>4, "abdfg"=>5, "abdefg"=>6, "acf"=>7, "abcdefg"=>8, "abcdfg"=>9)
    
    for i in 1:4
        newString = []
        for c in output[i]
            push!(newString, decoder[first(c)]) #concatenate next char to array
        end
        numbers[i] = reference[join(sort(newString))]
    end
    return numbers
end
    

decodeOutput (generic function with 1 method)

### Test decoder on single string

In [115]:
reference = "abcefg cf acdeg acdfg bcdf abdfg abdefg acf abcdefg abcdfg"

"abcefg cf acdeg acdfg bcdf abdfg abdefg acf abcdefg abcdfg"

In [88]:
testPattern ="acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab"
testOutput = "cdfeb fcadb cdfeb cdbaf"

"cdfeb fcadb cdfeb cdbaf"

In [98]:
testDecoder=buildDecoder(split(testPattern))

SubString{String}

Dict{Char,Char} with 7 entries:
  'f' => 'd'
  'g' => 'e'
  'a' => 'c'
  'c' => 'g'
  'd' => 'a'
  'e' => 'b'
  'b' => 'f'

In [114]:
decodeOutput(split(testOutput),testDecoder)

4-element Array{Int64,1}:
 5
 3
 5
 3

### Decode Output

In [123]:
nLines = size(signalPattern)[2]
newNumbers = Array{Int}(undef,nLines)

for i in 1:nLines
    decoder = buildDecoder(signalPattern[:,i])
    newNumber = decodeOutput(signalOutput[:,i],decoder)
    newNumbers[i] = parse(Int,join(string.(newNumber)))
end
newNumbers

200-element Array{Int64,1}:
 1989
 9441
  447
 4177
 1177
 4772
 5282
 1774
  881
 4127
 2855
 9184
 2921
    ⋮
 2451
 4747
 4871
 8877
 8864
 1494
 4721
 7310
 7937
 6181
 3194
 3709

In [124]:
sum(newNumbers)

1011284